<a href="https://colab.research.google.com/github/dhdbsrlw/Instruct-Tune-LLaMA-with-PEFT-Techniques/blob/main/DL_Training_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 초기 설정

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# !git clone https://github.com/Lightning-AI/lit-llama.git

Cloning into 'lit-llama'...
remote: Enumerating objects: 1925, done.
remote: Counting objects: 100% (664/664), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 1925 (delta 566), reused 510 (delta 495), pack-reused 1261
Receiving objects: 100% (1925/1925), 1.64 MiB | 5.54 MiB/s, done.
Resolving deltas: 100% (1201/1201), done.


In [ ]:
!git clone https://github.com/dhdbsrlw/Instruct-Tune-LLaMA-with-PEFT-Techniques.git

#**파이프라인**

In [ ]:
# %cd /content/drive/MyDrive/lit-llama

/content/drive/MyDrive/lit-llama


In [ ]:
%cd /content/drive/MyDrive/Instruct-Tune-LLaMA-with-PEFT-Techniques

In [ ]:
!pip install -r requirements.txt -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00


In [ ]:
!git-lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
!pip install tqdm

## STEP 1. 훈련 데이터셋 준비 (전처리)

한 번 데이터 저장되었으므로, 다시 실행시킬 필요 없다.

### 1-1. OpenLLaMA 모델 체크포인트 받아오기

In [ ]:
%pwd

'/content/drive/MyDrive/lit-llama'

In [ ]:
# 모델 체크포인트 받아오기 (실제 LLaMA 혹은 OpenLLaMA)
# 일단 OpenLLaMA 채택

# Make sure you have git-lfs installed (https://git-lfs.com): git lfs install
! git clone https://huggingface.co/openlm-research/open_llama_7b checkpoints/open-llama/7B


Cloning into 'checkpoints/open-llama/7B'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), 7.72 KiB | 28.00 KiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 5.42 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/lit-llama/checkpoints/open-llama/7B/.git/hooks/post-checkout': Permission denied
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/open-llama/7B --model_size 7B

In [ ]:
# Memory 이슈로 더 작은 모델 받아오기
# !git clone https://huggingface.co/openlm-research/open_llama_3b checkpoints/open-llama/3B

In [ ]:
# !python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/open-llama/3B --model_size 3B

### 1-2. 훈련 데이터셋 Prompt 전처리 및 Split

In [ ]:
%cd /content/drive/MyDrive/lit-llama

/content/drive/MyDrive/lit-llama


In [ ]:
%pwd

'/content/drive/MyDrive/lit-llama'

In [ ]:
!python scripts/prepare_dolly.py

# 전처리 완료된 데이터셋이 어느 경로에 저장되는지 살펴보기
# (분할 전) 전체 데이터셋은 - lit-llama/data/dolly 폴더 내 저장
# (분할 후) 데이터셋은 - lit-llama/data 폴더 내 .pt 형식으로 저장

train has 13,011 samples
val has 2,000 samples
Processing train split ...
100% 13011/13011 [00:14<00:00, 870.90it/s]
Processing test split ...
100% 2000/2000 [00:02<00:00, 848.11it/s]


## STEP 2. 파인튜닝

### 2-1. wandb 연동

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### 3-2. 파인튜닝 수행

#### `common_1` config

In [ ]:
# 본격적으로 파인튜닝 시키기 전에, WANDB 와 연결
# 파인튜닝 코드 args 살펴보고, 적절히 설정 (pretrained_path 수정 필요)

!python finetune/lora.py --data_dir data/dolly --out_dir out/lora/common1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 70% 11310/16260 [1:04:30<26:45,  3.08it/s]iter 11310: loss 5.5753, time: 424.54ms
 70% 11311/16260 [1:04:30<29:16,  2.82it/s]iter 11311: loss 5.2879, time: 220.29ms
 70% 11312/16260 [1:04:30<25:58,  3.18it/s]iter 11312: loss 4.3855, time: 214.95ms
 70% 11313/16260 [1:04:31<23:31,  3.50it/s]iter 11313: loss 5.5795, time: 376.93ms
 70% 11314/16260 [1:04:31<25:49,  3.19it/s]iter 11314: loss 5.4654, time: 315.87ms
 70% 11315/16260 [1:04:31<25:54,  3.18it/s]iter 11315: loss 4.4411, time: 151.27ms
 70% 11316/16260 [1:04:31<21:54,  3.76it/s]iter 11316: loss 6.0873, time: 464.06ms
 70% 11317/16260 [1:04:32<26:50,  3.07it/s]iter 11317: loss 4.8359, time: 236.22ms
 70% 11318/16260 [1:04:32<24:39,  3.34it/s]iter 11318: loss 5.1508, time: 275.15ms
 70% 11319/16260 [1:04:32<24:05,  3.42it/s]iter 11319: loss 5.5632, time: 284.32ms
 70% 11320/16260 [1:04:33<23:55,  3.44it/s]iter 11320: loss 4.8736, time: 209.30ms
 70% 11321/16260 [1:04:33<21:56,  3.75it/s]iter 113

In [ ]:
# adapter-v1
!python finetune/adapter.py --data_dir data/dolly --out_dir out/adapter/common1

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231206_080013-2k6eibc9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run common_1
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/adapter
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/adapter/runs/2k6eibc9
/content/drive/MyDrive/lit-llama/finetune/adapter.py:283: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter.py:283: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-reference.

In [ ]:
# adapter-v2
!python finetune/adapter_v2.py --data_dir data/dolly --out_dir out/adapter_v2/common1

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231206_093317-upgwn7i5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run common_1
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/adapter_v2
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/adapter_v2/runs/upgwn7i5
/content/drive/MyDrive/lit-llama/finetune/adapter_v2.py:289: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2.py:289: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#ap

In [ ]:
# full-layer
!python finetune/full.py --data_dir data/dolly --out_dir out/full/common1

/content/drive/MyDrive/lit-llama/finetune/full.py:230: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/full.py:230: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-reference.
    Importing from jsonargparse.cli is kept only to avoid breaking code that does not correctly use the public
    API. It will no longer be available from v5.0.0.

  from jsonargparse.cli import CLI
/usr/local/lib/python3.10/dist-packages/lightning/fabric/strategies/fsdp.py:703: `FSDPStrategy(activation_checkpointing=<class 'lit_llama.model.Block'>)` is deprecated, use `FSDPStrategy(activation_checkpointing_policy={<class 'lit_llama.model.Bl

일단, base configuration 으로 훈련을 시켜놓고 해당 모델 체크포인트들을 저장해둔다. \
base 에서 improved 된 버전으로 모델 네트워크를 수정한다.

### `common_2` config

*   batch_size = 64
*   num_epochs = 3
*   block_size = 512
*   (data) max_seq_length = 512





In [ ]:
# full-layer
!python finetune/full.py --data_dir data/dolly --out_dir out/full/common2

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_012414-fiwq5duo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run common_2_full
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/fiwq5duo
/content/drive/MyDrive/lit-llama/finetune/full.py:246: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/full.py:246: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-reference.


In [ ]:
# adapter-v1
!python finetune/adapter.py --data_dir data/dolly --out_dir out/adapter/sparse_1

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_093321-zpip1rao
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sparse_1
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/1207
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/1207/runs/zpip1rao
/content/drive/MyDrive/lit-llama/finetune/adapter.py:284: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter.py:284: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-reference.
    I

### `final-조건1` config

- model.py 는 수정할 필요 없음. \
- lit_llama 폴더 내 파일은 수정할 필요 없음.
- finetune 폴더 내에서 기존 파일의 llama 모델 import 경로와 config 설정을 수정해주어야 함.

In [ ]:
# adapter-v1-base
!python finetune/adapter.py --data_dir data/dolly --out_dir out/final/adapter_v1_base

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_134003-omxtmt3n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run adapter_v1_base
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/omxtmt3n
/content/drive/MyDrive/lit-llama/finetune/adapter.py:284: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter.py:284: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-ref

In [ ]:
# adapter-v2-base
!python finetune/adapter_v2.py --data_dir data/dolly --out_dir out/final/adapter_v２_base

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_134638-x26cgibn
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run adapter_v2_base
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/x26cgibn
/content/drive/MyDrive/lit-llama/finetune/adapter_v2.py:289: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2.py:289: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#a

In [ ]:
# adapter-v1-sparse
!python finetune/adapter_sparse.py --data_dir data/dolly --out_dir out/final/adapter_v1_sparse_local

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_140024-9b92kk1u
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run adapter_v1_sparse_local
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/9b92kk1u
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs

- wandb name, output file name, adpater_sparseAttn name 바꿔주어야 함.

In [ ]:
# adapter-v1-sparse
!python finetune/adapter_sparse.py --data_dir data/dolly --out_dir out/final/adapter_v1_sparse_all

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_140227-0gg3zc8n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run adapter_v1_sparse_all
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/0gg3zc8n
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.i

In [ ]:
# adapter-v1-sparse
!python finetune/adapter_sparse.py --data_dir data/dolly --out_dir out/final/adapter_v1_sparse_strided

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_140406-nt6o3ekz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run adapter_v1_sparse_strided
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/nt6o3ekz
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedo

strided 는 (코드 구현 상) 생략

In [ ]:
# adapter-v2-sparse
!python finetune/adapter_v2_sparse.py --data_dir data/dolly --out_dir out/final/adapter_v2_sparse_all

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_140746-13djdqnv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run adapter_v2_sparse_all
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/13djdqnv
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:289: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:289: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthe

In [ ]:
# adapter-v2-sparse
!python finetune/adapter_v2_sparse.py --data_dir data/dolly --out_dir out/final/adapter_v2_sparse_local

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_140850-u5unnrkt
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run adapter_v2_sparse_local
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/u5unnrkt
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:289: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:289: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readt

### `final-조건2` config

In [ ]:
# adapter-v1-base
!python finetune/adapter.py --data_dir data/dolly --out_dir out/final/c2/adapter_v1_base

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_141533-wmkbjg8k
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run c2_adapter_v1_base
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/wmkbjg8k
/content/drive/MyDrive/lit-llama/finetune/adapter.py:284: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter.py:284: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-

In [ ]:
# adapter-v1-sparse
!python finetune/adapter_sparse.py --data_dir data/dolly --out_dir out/final/c2/adapter_v1_sparse

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231207_150317-gp5lfdu7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run c2_adapter_v1_sparse_all
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/gp5lfdu7
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_sparse.py:284: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedoc

In [ ]:
# adapter-v2-base
!python finetune/adapter_v2.py --data_dir data/dolly --out_dir out/final/c2/adapter_v2_base

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231208_120729-erz0wbhd
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run c2_adapter_v2_base
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/erz0wbhd
/content/drive/MyDrive/lit-llama/finetune/adapter_v2.py:289: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2.py:289: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable

In [ ]:
# adapter-v2-sparse
!python finetune/adapter_v2_sparse.py --data_dir data/dolly --out_dir out/final/adapter_v2_ablation

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231208_170425-a6nwy73d
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ablation_c2_adapter_v2_sparse
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/a6nwy73d
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:290: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:290: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse

In [ ]:
# adapter-v2-sparse-dropout
!python finetune/adapter_v2_sparse_drop.py --data_dir data/dolly --out_dir out/final/c2/adapter_v2_sparse_dropout

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231208_131544-nh1w2ion
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run c2_adapter_v2_sparse_all_dropout
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/nh1w2ion
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse_drop.py:290: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse_drop.py:290: JsonargparseDeprecationWarning: 
    Only use the public API as described in https:/

In [ ]:
# lora-base
!python finetune/lora.py --data_dir data/dolly --out_dir out/final/c2/lora_base

새로운 실험

In [ ]:
# adapter-v2-sparse
!python finetune/adapter_v2_sparse.py --data_dir data/dolly --out_dir out/final/adapter_v2_ablation_2

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231209_041756-sym2zw11
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ablation_c2_adapter_v2_sparse
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/COSE474
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/COSE474/runs/sym2zw11
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:290: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/adapter_v2_sparse.py:290: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse

#**STEP3. Evaluation**

In [ ]:
# adapter-base
!python evaluate/adapter.py --adapter_path /content/drive/MyDrive/lit-llama/out/final/c2/adapter_v1_base/lit-llama-adapter-finetuned.pth --quantize llm.int8

Loading model ...
Time to load model: 6.86 seconds.
Generating test split: 100% 4358/4358 [00:00<00:00, 43467.39 examples/s]
Generating train split: 100% 36718/36718 [00:00<00:00, 50644.87 examples/s]
Generating validation split: 100% 3760/3760 [00:00<00:00, 51568.02 examples/s]
100% 157/157 [06:07<00:00,  2.34s/it]
torch.Size([1, 321472]) torch.Size([1984, 32000])
Perplexity on wikitext: 32000.07
Extracting data files: 100% 3/3 [00:00<00:00, 2148.35it/s]
Generating train split: 100% 42068/42068 [00:00<00:00, 57043.89 examples/s]
Generating test split: 100% 3761/3761 [00:00<00:00, 57476.74 examples/s]
Generating validation split: 100% 3370/3370 [00:00<00:00, 58962.33 examples/s]
100% 53/53 [02:02<00:00,  2.31s/it]
torch.Size([1, 107447]) torch.Size([951, 32000])
Perplexity on ptb: 32000.07
/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was 

In [ ]:
# adapter-v1-sparse
!python evaluate/adapter_sparse.py --adapter_path out/final/c2/adapter_v1_sparse/lit-llama-adapter-finetuned.pth --quantize llm.int8

Loading model ...
Time to load model: 4.95 seconds.
100% 157/157 [06:14<00:00,  2.39s/it]
torch.Size([1, 321472]) torch.Size([1984, 32000])
Perplexity on wikitext: 32000.07
100% 53/53 [02:05<00:00,  2.36s/it]
torch.Size([1, 107447]) torch.Size([951, 32000])
Perplexity on ptb: 32000.07
/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
100% 256/256 [10:10<00:00,  2.38s/it]
torch.Size([1, 524288]) torch.Size([2048, 32000])
Perplexity on c4: 32000.07


Time for inference: 610.45 sec total, 1560.71 tokens/sec
Memory used: 11.41 GB
